In [1]:
import pandas as pd

data_path = './comment_code_sol.pkl'


In [2]:
dataset = pd.read_pickle(data_path)

In [5]:
# drop all code and comment value not longer as 20 characters
print("INFO: Original dataset length:", len(dataset))

dataset=dataset[dataset['code_string'].str.len() >= 20]
print("INFO: dataset length after code length refinment:", len(dataset))

dataset=dataset[dataset['comments'].str.len() >= 20]
print("INFO: dataset length after comment length refinment:", len(dataset))


INFO: Original dataset length: 17211242
INFO: dataset length after code length refinment: 13728139
INFO: dataset length after comment length refinment: 12650850


In [26]:
# remove lines with no code
def hasMarker(code):
    if ';' in code:
        return True
    
    elif '}' in code:
        return True
    
    elif '{' in code:
        return True
    
    elif ';' in code:
        return True
    else:
        return False

dataset=dataset[dataset['code_string'].apply(lambda x: hasMarker(x))]
print("INFO: dataset length after marker processing:", len(dataset))
 
    

INFO: dataset length after marker processing: 12332748


In [35]:
# Drop duplicate on code value
dataset = dataset.drop_duplicates(subset=['code_string'], keep='first')
print("INFO: dataset length after dropping duplicates:", len(dataset))


INFO: dataset length after dropping duplicates: 1359405


In [6]:
dataset

,file_name,comments,code_string
0,// SPDX-License-Identifier: MIT\npragma solidi...,Inspired by PIXL and Chubbies.,"contract DumplingERC721 is ERC721, Ownable, Pr..."
1,// SPDX-License-Identifier: MIT\npragma solidi...,SPDX-License-Identifier: MIT,pragma solidity ^0.7.4;\n
2,// SPDX-License-Identifier: MIT\npragma solidi...,* @dev Withdraw ether from this contract (Call...,function withdraw() onlyOwner public {\n ...
3,// SPDX-License-Identifier: MIT\npragma solidi...,"_setTokenURI(newTokenId, Strings.toString(new...",function steamDumplings(uint256 numDumplin...
4,// SPDX-License-Identifier: MIT\npragma solidi...,"_setTokenURI(newTokenId, Strings.toString(new...",function steamDumplings(uint256 numDumplin...
...,...,...,...
17211237,./partial_match/77/0x925aFFbA0902cC94d103a1CF5...,* @dev Give your vote (including votes delegat...,function vote(uint proposal) public {\n ...
17211238,./partial_match/77/0x925aFFbA0902cC94d103a1CF5...,* @dev Computes the winning proposal taking al...,function winningProposal() public view\n ...
17211239,./partial_match/77/0x925aFFbA0902cC94d103a1CF5...,* @dev Computes the winning proposal taking al...,function winningProposal() public view\n ...
17211240,./partial_match/77/0x925aFFbA0902cC94d103a1CF5...,* @dev Computes the winning proposal taking al...,function winningProposal() public view\n ...


In [49]:
i = 6
print(dataset.comments.iloc[i])
print('-'*100)
print(dataset.code_string.iloc[i])

* @dev See {IERC721-balanceOf}./
----------------------------------------------------------------------------------------------------
    function balanceOf(address owner) public view override returns (uint256) {
        require(owner != address(0), "ERC721: balance query for the zero address");

        return _holderTokens[owner].length();
    }




In [41]:
dataset.to_pickle('./filtered_comment_code_sol.pkl')